# Speech recognition + NLTK

The goal of this repo is to show an exploration of the `NLTK` Python library for Natural Language Processing. We'll test various functions and create a program that responds to questions using human voice and the Google Speech Recognition API.

# I- Google Speech Recognition

First, let's test the Google API. 

In [2]:
import speech_recognition as sr

In [8]:
r = sr.Recognizer()
with sr.Microphone() as source:
    print("Say something!")
    audio = r.listen(source, timeout=8) #listening for 8 seconds
    print(r.recognize_google(audio))

Say something!
hello my name is this is a test or you listening to me


Youhou, there is a little mistake due to my spelling byt now we know that it works ! Let's launch our next step. 

# II- Speech Recognition

Now, the goal is to define some scenarios in which a robot hears an instruction like "Tell me a joke" and responds by actually telling a joke, for instance. 

I will do that by : 

- Make my computer record my audio for 10s 
- With `NLTK`, tokenize the sentences and proceed to some stemming
- If there are the stems of `"joke"` and of `"telling"` then the robot tells a joke. 
- I will just code an exception if there is also `"no"` just near the stem of `"telling"`, let's say like 2 or three words apart. 

And that's it, let's code that. I will choose the joke myself but we could imagine implementing **more complex decision trees** and **plugging an LLM** on our little system. 

- I will use the `Porter` stemming algorithm. 
- First, let's set up our environment by loading some modules and defining a cool function. 

In [14]:
import nltk 
nltk.download('punkt') #downloading library
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Initialize the NLTK Porter Stemmer
stemmer = PorterStemmer()

# Function to check if a list of words contains a stem
def contains_stem(word_list, target_stem):
    for word in word_list:
        if stemmer.stem(word) == target_stem:
            return True
    return False

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hugocaetano/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Now, let's investigate on the stems we need. 

In [18]:
#First getting words
text = "My name was Hugo. I did not want you to telling me a joking"
words = word_tokenize(text)
print("words = ", words)

#Then extracting stems
stems = [stemmer.stem(word) for word in words]
print("stems = ", stems)

words =  ['My', 'name', 'was', 'Hugo', '.', 'I', 'did', 'not', 'want', 'you', 'to', 'telling', 'me', 'a', 'joking']
stems =  ['my', 'name', 'wa', 'hugo', '.', 'i', 'did', 'not', 'want', 'you', 'to', 'tell', 'me', 'a', 'joke']


Nice, as we can see, "telling" becomes "tell", and there are other changes. I think i'm ready to code the little script. 

In [20]:
#Instantiating a joke
joke = "Quel cri fait un quoicoumouton ? Quoicoubêêêêêêh"

# Initialize the SpeechRecognizer
r = sr.Recognizer()

# Record audio for 10 seconds
with sr.Microphone() as source:
    print("Say something!")
    audio = r.listen(source, timeout=15) #here we can change the demo time

# Convert audio to text using Google Speech Recognition
audio_text = r.recognize_google(audio)
print("You said: " + audio_text)

# Tokenize and stem the words in the audio text
words = word_tokenize(audio_text.lower())  # Convert to lowercase for case insensitivity
stems = [stemmer.stem(word) for word in words]

# Check if "joke" and "telling" stems are present
if contains_stem(stems, "joke") and contains_stem(stems, "tell"):
    # Check for the presence of "no" within 2 or 3 words of "telling"
    no_index = stems.index("no") if "no" in stems else -1
    tell_index = stems.index("tell") if "tell" in stems else -1
    if no_index != -1 and abs(no_index - tell_index) <= 4: #I think 4 is more relevant. For cases like "I do not want you to tell a joke."
        print("I won't tell a joke.")
    else:
        # Let's tell a joke now
        print(f"Sure, here's a joke: {joke}")
else:
    print("I didn't understand your instruction.")


Say something!
You said: hello my name is Google please tell a joke
Sure, here's a joke: Quel cri fait un quoicoumouton ? Quoicoubêêêêêêh


Well it seems to work pretty well ! This notebook was just created to do a little experiment, but one could imagine setting up complex sets of rules and plugging lots of cool tools on that simple base ! 